# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

In [1]:
import nltk
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB

## Загрузка и первичный анализ данных

In [2]:
try:
    df_comments = pd.read_csv('toxic_comments.csv', index_col=0)
except:
    df_comments = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
print(df_comments.info())
df_comments.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Посмотрим на наличие пропусков и дубликатов

In [3]:
print("Количество дубликатов =", df_comments.duplicated().sum())
print("Количество пропусков", df_comments.isna().sum(), sep='\n')

Количество дубликатов = 0
Количество пропусков
text     0
toxic    0
dtype: int64


Посмотрим на представленные классы

In [4]:
df_comments['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Видим сильный дисбаланс классов, можем для ускорения вычислений ограничим наши данные 50000 объетов

In [5]:
df_comments_cut = df_comments.head(50000)

Напишем функцию для обработки комментария

In [6]:
def process_comment(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text.lower())
    stop_words_set = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    text = [stemmer.stem(word) for word in tokens if word not in stop_words_set]
    return ' '.join(text)

Получим новый столбец с уже обработанным текстом

In [7]:
df_comments_cut.loc[:, 'text_stem'] = df_comments_cut.loc[:, 'text'].apply(process_comment)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Посмотрим на полученный текст

In [8]:
df_comments_cut.head(10)

,text,toxic,text_stem
0,Explanation\nWhy the edits made under my usern...,0,explan edit made usernam hardcor metallica fan...
1,D'aww! He matches this background colour I'm s...,0,daww match background colour im seem stuck tha...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im realli tri edit war guy constant re...
3,"""\nMore\nI can't make any real suggestions on ...",0,cant make real suggest improv wonder section s...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chanc rememb page that
5,"""\n\nCongratulations from me as well, use the ...",0,congratul well use tool well talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksuck piss around work
7,Your vandalism to the Matt Shirvington article...,0,vandal matt shirvington articl revert pleas do...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorri word nonsens offens anyway im intend wri...
9,alignment on this subject and which are contra...,0,align subject contrari dulithgow


## Подбор модели

In [9]:
RANDOM_STATE = 42
final_p = Pipeline(steps=[
    ('preprocessing', TfidfVectorizer(min_df=5)),
    ('model', LogisticRegression())
])

Воспользуемся 3 стандартными моделями

In [10]:
param_grid = [
    {
        'model': [
            LogisticRegression(random_state=RANDOM_STATE),
            SVC(kernel='linear', probability=True, random_state=RANDOM_STATE),
            ComplementNB(),
        ]
    }
]

В качестве метрики для сравнения используем f1

In [11]:
grid_search = GridSearchCV(
    final_p,
    param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
)

Разделим выборку на тренировочную и тестовую

In [12]:
X = df_comments_cut['text_stem']
y = df_comments_cut['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=RANDOM_STATE)

Используем Tf-idf для кодирования текста

In [13]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

Используем grid search для подбора оптимальной модели

In [14]:
grid_search.fit(X_train, y_train)
best_model_classification = grid_search.best_estimator_

In [15]:
print('Лучшая модель и её параметры:\n', best_model_classification)
print('Метрика модели на тренировочных данных:', grid_search.best_score_)

Лучшая модель и её параметры:
 Pipeline(steps=[('preprocessing', TfidfVectorizer(min_df=5)),
                ('model',
                 SVC(kernel='linear', probability=True, random_state=42))])
Метрика модели на тренировочных данных: 0.7381452752687793


Посмотрим на результаты остальных моделей

In [16]:
results_df = pd.DataFrame(grid_search.cv_results_)
sorted_results = results_df.sort_values(by='mean_test_score', ascending=False)
sorted_results = sorted_results[['params', 'mean_test_score']]
sorted_results

,params,mean_test_score
1,"{'model': SVC(kernel='linear', probability=Tru...",0.738145
0,{'model': LogisticRegression(random_state=42)},0.668011
2,{'model': ComplementNB()},0.633265


## Проверка модели на тесте

In [17]:
svc_model = SVC(kernel='linear')
svc_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [18]:
y_pred = svc_model.predict(X_test_tfidf)
f1_score(y_pred, y_test)

0.7624309392265193

Как видим результат соответствует требованиям

## Вывод
В ходе этой работы мы предобработали данные, выбрали и обучили модель для определния токичности комментария.
* Было решено использовать не все данные, а только некоторую часть для ускорения обучения моделей.Для подготовки текста перед подачей в модель мы удалили знаки препинания в каждом предложении, перевили всё в нижний регистр, а с также воспользовались SnowballStemmer для обрезки слов. Для векторизации предложениий мы использовали Tf-Idf vectorizer.
* Для подбора наилучшей модели мы использовали GridSerch, а оценивались они по метрике f1. Входе моделирования мы выяснили, что лучшие результвты даёт SVC модель и проверили её на тестовой выборке, полученные результат в f1=0.75 полностью соответствует заданным требованиям.

Для классификации комментариев предлагаем использовать модель SVC с предварительной обработкой и Tf-idf векторизацией.